In [33]:
# imports
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import make_scorer


from helper_functions import load_data
from helper_functions import my_split
from helper_functions import downsample_majority
from helper_functions import model_prep
from helper_functions import get_results
from helper_functions import get_f1

In [2]:
# loading data
df = load_data()
df.head()

,name,desc,goal,disable_communication,country,currency,deadline,launched_at,final_status,campaign_length,launch_year,launch_month,launch_day,launch_weekday
0,drawing for dollars,I like drawing pictures. and then i color them...,20.0,0,US,USD,2009-05-03 02:59:59,2009-04-24 15:52:03,1,8,2009,4,24,4
1,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,0,US,USD,2009-05-15 19:10:00,2009-04-28 23:26:32,0,17,2009,4,28,1
2,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,0,US,USD,2009-05-22 17:26:00,2009-05-12 17:39:58,0,10,2009,5,12,1
3,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,0,US,USD,2009-05-28 20:09:00,2009-04-28 20:58:50,1,30,2009,4,28,1
4,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,0,US,USD,2009-05-31 07:38:00,2009-05-01 08:22:21,0,30,2009,5,1,4


In [3]:
# splitting data into test and train
year = 2015
train, test = my_split(df, year)

In [4]:
# selecting features and target
features = ['goal', 
        'campaign_length', 
        'launch_month', 
        'launch_day', 
        'launch_weekday', 
        'disable_communication',
        'country', 
        'currency']

target = 'final_status'
epochs = 20

In [5]:
# basic neural network - no change to class balance
X_train, y_train, X_test, y_test = model_prep(train, test, features, target)

input_dim = X_train.shape[1]

model = Sequential()
model.add(Dense(10, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', get_f1]
)

history = model.fit(x=X_train, 
          y=y_train, 
          epochs=epochs,
          validation_data=(X_test, y_test) 
          )

# scores with no change
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_true = y_test

get_results(y_true, y_pred)

Epoch 1/20
2826/2826 [==============================] - 11s 4ms/step - loss: 0.6488 - accuracy: 0.6550 - get_f1: 0.0692 - val_loss: 0.5915 - val_accuracy: 0.7609 - val_get_f1: 0.0343
Epoch 2/20
2826/2826 [==============================] - 9s 3ms/step - loss: 0.6273 - accuracy: 0.6626 - get_f1: 0.0242 - val_loss: 0.5822 - val_accuracy: 0.7602 - val_get_f1: 0.0404
Epoch 3/20
2826/2826 [==============================] - 9s 3ms/step - loss: 0.6248 - accuracy: 0.6634 - get_f1: 0.0403 - val_loss: 0.5743 - val_accuracy: 0.7607 - val_get_f1: 0.0315
Epoch 4/20
2826/2826 [==============================] - 9s 3ms/step - loss: 0.6228 - accuracy: 0.6637 - get_f1: 0.0475 - val_loss: 0.5818 - val_accuracy: 0.7514 - val_get_f1: 0.0854
Epoch 5/20
2826/2826 [==============================] - 9s 3ms/step - loss: 0.6222 - accuracy: 0.6636 - get_f1: 0.0523 - val_loss: 0.5835 - val_accuracy: 0.7463 - val_get_f1: 0.1142
Epoch 6/20
2826/2826 [==============================] - 10s 3ms/step - loss: 0.6220 - acc

In [45]:
# basic neural network - downsampling minority class
train_downsampled = downsample_majority(train)

X_train, y_train, X_test, y_test = model_prep(train_downsampled, test, features, target)

input_dim = X_train.shape[1]

model = Sequential()
model.add(Dense(10, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', get_f1]
)

history = model.fit(x=X_train, 
          y=y_train, 
          epochs=epochs,
          validation_data=(X_test, y_test) 
          )

# scores with upsampling
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_true = y_test

get_results(y_true, y_pred)

Epoch 1/20
1905/1905 [==============================] - 5s 2ms/step - loss: 0.6908 - accuracy: 0.5423 - get_f1: 0.5814 - val_loss: 0.7210 - val_accuracy: 0.4191 - val_get_f1: 0.3685
Epoch 2/20
1905/1905 [==============================] - 4s 2ms/step - loss: 0.6804 - accuracy: 0.5561 - get_f1: 0.5894 - val_loss: 0.7253 - val_accuracy: 0.4057 - val_get_f1: 0.3820
Epoch 3/20
1905/1905 [==============================] - 5s 3ms/step - loss: 0.6773 - accuracy: 0.5651 - get_f1: 0.5837 - val_loss: 0.7408 - val_accuracy: 0.4059 - val_get_f1: 0.3854
Epoch 4/20
1905/1905 [==============================] - 4s 2ms/step - loss: 0.6763 - accuracy: 0.5672 - get_f1: 0.5829 - val_loss: 0.7361 - val_accuracy: 0.4030 - val_get_f1: 0.3861
Epoch 5/20
1905/1905 [==============================] - 4s 2ms/step - loss: 0.6751 - accuracy: 0.5724 - get_f1: 0.5843 - val_loss: 0.7302 - val_accuracy: 0.4151 - val_get_f1: 0.3879
Epoch 6/20
1905/1905 [==============================] - 4s 2ms/step - loss: 0.6744 - accur

In [42]:
# basic neural network - downsampling majority class
train_downsampled = downsample_majority(train)
X_train, y_train, X_test, y_test = model_prep(train_downsampled, test, features, target)

input_dim = X_train.shape[1]

model = Sequential()
model.add(Dense(10, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', get_f1]
)

history = model.fit(x=X_train, 
          y=y_train, 
          epochs=epochs,
          validation_data=(X_test, y_test) 
          )

# scores with downsampling
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_true = y_test

get_results(y_true, y_pred)

Epoch 1/20
1905/1905 [==============================] - 7s 3ms/step - loss: 0.6944 - accuracy: 0.5494 - get_f1: 0.5791 - val_loss: 0.7307 - val_accuracy: 0.3903 - val_get_f1: 0.3795
Epoch 2/20
1905/1905 [==============================] - 6s 3ms/step - loss: 0.6802 - accuracy: 0.5620 - get_f1: 0.5946 - val_loss: 0.7328 - val_accuracy: 0.4019 - val_get_f1: 0.3806
Epoch 3/20
1905/1905 [==============================] - 6s 3ms/step - loss: 0.6778 - accuracy: 0.5658 - get_f1: 0.5933 - val_loss: 0.7419 - val_accuracy: 0.4155 - val_get_f1: 0.3788
Epoch 4/20
1905/1905 [==============================] - 5s 3ms/step - loss: 0.6761 - accuracy: 0.5698 - get_f1: 0.5930 - val_loss: 0.7190 - val_accuracy: 0.4320 - val_get_f1: 0.3862
Epoch 5/20
1905/1905 [==============================] - 6s 3ms/step - loss: 0.6745 - accuracy: 0.5758 - get_f1: 0.5955 - val_loss: 0.7152 - val_accuracy: 0.4464 - val_get_f1: 0.3865
Epoch 6/20
1905/1905 [==============================] - 5s 3ms/step - loss: 0.6735 - accur

In [43]:
# adding layers - downsampling majority class
train_downsampled = downsample_majority(train)

X_train, y_train, X_test, y_test = model_prep(train_downsampled, test, features, target)

input_dim = X_train.shape[1]

model = Sequential()
model.add(Dense(30, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', get_f1]
)

# fitting the model
history = model.fit(x=X_train, 
          y=y_train, 
          epochs=epochs,
          validation_data=(X_test, y_test) 
          )

# scores
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_true = y_test

get_results(y_true, y_pred)

Epoch 1/20
1905/1905 [==============================] - 10s 5ms/step - loss: 0.6878 - accuracy: 0.5434 - get_f1: 0.5582 - val_loss: 0.6948 - val_accuracy: 0.4720 - val_get_f1: 0.3654
Epoch 2/20
1905/1905 [==============================] - 10s 5ms/step - loss: 0.6827 - accuracy: 0.5550 - get_f1: 0.5562 - val_loss: 0.7184 - val_accuracy: 0.4205 - val_get_f1: 0.3736
Epoch 3/20
1905/1905 [==============================] - 10s 5ms/step - loss: 0.6808 - accuracy: 0.5599 - get_f1: 0.5473 - val_loss: 0.7180 - val_accuracy: 0.4165 - val_get_f1: 0.3850
Epoch 4/20
1905/1905 [==============================] - 9s 5ms/step - loss: 0.6788 - accuracy: 0.5643 - get_f1: 0.5578 - val_loss: 0.7089 - val_accuracy: 0.4296 - val_get_f1: 0.3859
Epoch 5/20
1905/1905 [==============================] - 10s 5ms/step - loss: 0.6767 - accuracy: 0.5706 - get_f1: 0.5630 - val_loss: 0.6972 - val_accuracy: 0.4978 - val_get_f1: 0.3730
Epoch 6/20
1905/1905 [==============================] - 13s 7ms/step - loss: 0.6745 - 

In [44]:
# adding layers, massive amount of neurons - downsampling majority class
train_downsampled = downsample_majority(train)

X_train, y_train, X_test, y_test = model_prep(train_downsampled, test, features, target)

input_dim = X_train.shape[1]

model = Sequential()
model.add(Dense(300, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', get_f1]
)

# fitting the model
history = model.fit(x=X_train, 
          y=y_train, 
          epochs=epochs,
          validation_data=(X_test, y_test) 
          )

# scores
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_true = y_test

get_results(y_true, y_pred)

Epoch 1/20
1905/1905 [==============================] - 51s 27ms/step - loss: 0.6846 - accuracy: 0.5539 - get_f1: 0.5439 - val_loss: 0.7139 - val_accuracy: 0.4464 - val_get_f1: 0.3681
Epoch 2/20
1905/1905 [==============================] - 36s 19ms/step - loss: 0.6788 - accuracy: 0.5696 - get_f1: 0.5611 - val_loss: 0.7199 - val_accuracy: 0.4698 - val_get_f1: 0.3730
Epoch 3/20
1905/1905 [==============================] - 24s 12ms/step - loss: 0.6764 - accuracy: 0.5740 - get_f1: 0.5704 - val_loss: 0.7278 - val_accuracy: 0.4607 - val_get_f1: 0.3757
Epoch 4/20
1905/1905 [==============================] - 24s 13ms/step - loss: 0.6745 - accuracy: 0.5779 - get_f1: 0.5672 - val_loss: 0.7304 - val_accuracy: 0.4421 - val_get_f1: 0.3877
Epoch 5/20
1905/1905 [==============================] - 24s 13ms/step - loss: 0.6731 - accuracy: 0.5816 - get_f1: 0.5875 - val_loss: 0.7041 - val_accuracy: 0.4591 - val_get_f1: 0.3912
Epoch 6/20
1905/1905 [==============================] - 24s 13ms/step - loss: 0.

In [10]:
# adding layers, massive amount of neurons - downsampling majority class
train_downsampled = downsample_majority(train)

X_train, y_train, X_test, y_test = model_prep(train_downsampled, test, features, target)

input_dim = X_train.shape[1]

model = Sequential()
model.add(Dense(300, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(300,  activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(300,  activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(300,  activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', get_f1]
)

# fitting the model
history = model.fit(x=X_train, 
          y=y_train, 
          epochs=epochs,
          validation_data=(X_test, y_test) 
          )

# scores
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_true = y_test

get_results(y_true, y_pred)

Epoch 1/20
1905/1905 [==============================] - 43s 22ms/step - loss: 0.6832 - accuracy: 0.5587 - get_f1: 0.5592 - val_loss: 0.7247 - val_accuracy: 0.4274 - val_get_f1: 0.3665
Epoch 2/20
1905/1905 [==============================] - 42s 22ms/step - loss: 0.6772 - accuracy: 0.5728 - get_f1: 0.5757 - val_loss: 0.8123 - val_accuracy: 0.3982 - val_get_f1: 0.3861
Epoch 3/20
1905/1905 [==============================] - 47s 25ms/step - loss: 0.6747 - accuracy: 0.5761 - get_f1: 0.5748 - val_loss: 0.7350 - val_accuracy: 0.4726 - val_get_f1: 0.3705
Epoch 4/20
1905/1905 [==============================] - 43s 23ms/step - loss: 0.6738 - accuracy: 0.5797 - get_f1: 0.5920 - val_loss: 0.7336 - val_accuracy: 0.4725 - val_get_f1: 0.3727
Epoch 5/20
1905/1905 [==============================] - 43s 22ms/step - loss: 0.6723 - accuracy: 0.5819 - get_f1: 0.5904 - val_loss: 0.7481 - val_accuracy: 0.4266 - val_get_f1: 0.3899
Epoch 6/20
1905/1905 [==============================] - 48s 25ms/step - loss: 0.

In [46]:
# grid search, this will return accuracy but that should hopefully also equal a higher f1
# when trained on a balanced set
train_downsampled = downsample_majority(train)
X_train, y_train, X_test, y_test = model_prep(train_downsampled, test, features, target)
input_dim = X_train.shape[1]

def create_model(lr, optimizer='adam', number_layers=1,
        neurons1=1, act1='relu', init_mode1='uniform', dropout_rate1=0.0,
        neurons2=1, act2='relu', init_mode2='uniform', dropout_rate2=0.0,
        neurons3=1, act3='relu', init_mode3='uniform', dropout_rate3=0.0,
        neurons4=1, act4='relu', init_mode4='uniform', dropout_rate4=0.0,
        neurons5=1, act5='relu', init_mode5='uniform', dropout_rate5=0.0,
        init_mode6='uniform'):
    adam = Adam(learning_rate=lr)
    model = Sequential()
    if number_layers <= 1:
        model.add(Dense(neurons1, activation=act1, kernel_initializer=init_mode1, input_shape=(input_dim,)))
        model.add(Dropout(dropout_rate1))

    if number_layers <= 2:
        model.add(Dense(neurons2, activation=act2, kernel_initializer=init_mode2))
        model.add(Dropout(dropout_rate2))

    if number_layers <= 3:
        model.add(Dense(neurons3, activation=act3, kernel_initializer=init_mode3))
        model.add(Dropout(dropout_rate3))

    if number_layers <= 4:
        model.add(Dense(neurons4, activation=act4, kernel_initializer=init_mode4))
        model.add(Dropout(dropout_rate4))

    if number_layers <= 5:
        model.add(Dense(neurons5, activation=act5, kernel_initializer=init_mode5))
        model.add(Dropout(dropout_rate5))

    model.add(Dense(1, activation='sigmoid', kernel_initializer=init_mode6))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=10)


param_grid = {'batch_size': [50, 100],
            'epochs': [20],
            'lr': [.01, .1, .5], 
            'optimizer': ['adam'], 
            'number_layers': [1, 2, 3], 
            'neurons1': [1, 5, 10, 20], 
            'act1': ['relu'], 
            'init_mode1': ['uniform'], 
            'dropout_rate1': [0, .1, .3],   
            'neurons2': [1, 5, 10, 20], 
            'act2': ['relu'], 
            'init_mode2': ['uniform'], 
            'dropout_rate2': [0, .1, .3],   
            'neurons3': [1, 5, 10, 20], 
            'act3': ['relu'], 
            'init_mode3': ['uniform'], 
            'dropout_rate3': [0, .1, .3],
            #'neurons4': [1, 5, 10, 20],   # commenting these out to reduce run time
            #'act4': ['relu'], 
            #'init_mode4': ['uniform'], 
            #'dropout_rate4': [0, .1, .3],
            #'neurons5': [1, 5, 10, 20], 
            #'act5': ['relu'], 
            #'init_mode5': ['uniform'], 
            #'dropout_rate5': [0, .1, .3],
            'init_mode6': ['uniform']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")